In [1]:
import os
os.chdir("c:/Users/Asus/Desktop/kidney_Disease_Classifier")
print(os.getcwd())


c:\Users\Asus\Desktop\kidney_Disease_Classifier


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path  ## things defined in the config.yaml have to return here

In [4]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml,create_directories

In [5]:
import cnnClassifier.constants as const

print(dir(const))


['CONFIG_FILE_PATH', 'PARAMS_FILE_PATH', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            
        )
        return data_ingestion_config

    

    

In [7]:
import os
import zipfile 
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self)-> str:
        """
         
        Fetch the data from url
        """
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading file from :[{dataset_url}] into :[{zip_download_dir}]")
            
            file_id = dataset_url.split("/")[-2]
            prefix ='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Download data from {dataset_url} into file{zip_download_dir}")
            
        except Exception as e:
            raise e
    ## helpful to unzip the zipped file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [12]:
## pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-01 11:58:40,984: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-01 11:58:40,989: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-01 11:58:40,990: INFO: common: created directory at: artifacts_root]
[2025-09-01 11:58:40,992: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-01 11:58:40,992: INFO: 2539216739: Downloading file from :[https://drive.google.com/file/d/10MMubDH0wG9NdUxuuRfBnmiBqItY8WhJ/view?usp=sharing] into :[artifacts/data_ingestion/data.zip]]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=10MMubDH0wG9NdUxuuRfBnmiBqItY8WhJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=10MMubDH0wG9NdUxuuRfBnmiBqItY8WhJ&confirm=t&uuid=de94b26a-b8cc-4797-8829-8c406c93c4c9
To: c:\Users\Asus\Desktop\kidney_Disease_Classifier\artifacts\data_ingestion\data.zip
100%|██████████| 1.63G/1.63G [06:32<00:00, 4.15MB/s]

[2025-09-01 12:05:17,756: INFO: 2539216739: Download data from https://drive.google.com/file/d/10MMubDH0wG9NdUxuuRfBnmiBqItY8WhJ/view?usp=sharing into fileartifacts/data_ingestion/data.zip]


In [ ]:
_